In [ ]:
from itables import init_notebook_mode
import pandas as pd
import numpy as np
import re
init_notebook_mode(all_interactive=True)
pd.options.display.max_rows=300

In [ ]:
with open('iomap.txt') as f:
    generator = (line.strip('\n').split(' ', maxsplit=3) for line in f.readlines())
lines = list(generator)

In [ ]:
class ParseIomap:
    ''' Convert MEGA65's iomap.txt to Pandas DataFrame '''
    db = 0
    
    def initial_trim(self):
        ''' perform initial trimming of leading and trailing spaces '''
        mask = self.db.Description.notnull()
        self.db.Description[mask] = self.db.Description[mask].apply(lambda x: x.strip(' '))

    def get_mask(self, mask_str):
        ''' Convert "Bit" string to integer mask (single value or interval) '''
        if mask_str.isnumeric():
            bit = int(mask_str)
            assert(bit <= 7)
            return int(1 << bit)
        else:
            mask = 0
            begin, end = sorted(map(int, mask_str.split('-')))
            for bit in range(end-begin+1):
                mask = mask | (1 << bit)
            return int(mask)
        
    def mangle_description(self, name):
        ''' create a rust variable name from description field '''
        name = name.lower()\
                .replace('(read only) returns the', '')\
                .replace('<=', 'le')\
                .replace('>', 'gt')\
                .replace('$', '0x')\
                .replace('/', '_slash_')\
                .replace('0=', '')\
                .replace('1=', '')\
                .replace('&', '_and_')\
                .translate({ord(i):None for i in '#$@~\\`\''})\
                .translate({ord(i):'.' for i in ',=(:;'})\
                .replace('!', '_')\
                .replace('-', '_')\
                .replace('!', '_')\
                .replace('/', '_')\
                .replace('__', '_')\
                .replace('__', '_')\
                .split('.')[0]\
                .strip('_')\
                .strip(' ')
        if name[:1].isdigit():
            words = name.split(' ')
            if len(words) > 1:
                words[0], words[-1] = words[-1], words[0]
            return '_'.join(words)
        name = name.replace(' ', '_')
        return name
    
    def make_long_names(self):
        ''' create descriptive (long) symbol name from Description '''    
        self.db['LongName'] = self.db['Description']
        mask = self.db.LongName.notnull()
        self.db.LongName[mask] = self.db.LongName[mask].apply(self.mangle_description)

    def split_address_range(self):
        ''' split address range to two columns, Lower and Upper '''
        self.db[['Addresses', 'Mask']] = self.db['Addresses'].str.split('.', 1, expand=True)
        self.db[['Lower', 'Upper']] = self.db['Addresses'].str.split('-', 1, expand=True)
        del self.db['Addresses']
        self.db['Lower'] = self.db['Lower'].str.strip('$ ')
        self.db['Upper'] = self.db['Upper'].str.strip('$ ')

    def detect_categories(self):
        ''' detect chip (~category) '''
        self.db.Symbol = self.db.Symbol.apply(lambda x: x.replace('SUMMARY:','')) # not a real category
        self.db[['Category', 'Symbol']] = self.db['Symbol'].str.split(':', 1, expand=True)
        self.db['Symbol'] = self.db['Symbol'].str.split('@').str[0]
        dictionary = {'ETH': 'ETHERNET'}
        self.db.Category.replace(dictionary, inplace = True)

    def remove_dashes(self):
        ''' Rust doesn't like '-' in constants and variables '''
        mask = self.db.Symbol.notnull()
        self.db.Symbol[mask] = self.db.Symbol[mask].apply(lambda x: x.replace('-', '_'))
        dictionary = {'VIC-II': 'vic2', 'VIC-III': 'vic3', 'VIC-IV': 'vic4'}
        self.db.Category.replace(dictionary, inplace = True)

    def name_masks(self):
        ''' masks may be explicitly named. If not, give them the symbol name '''
        self.db[['Symbol', 'MaskName']] = self.db['Symbol'].str.rsplit('!', 1, expand=True)
        mask = self.db.MaskName.isnull()
        self.db.MaskName[mask] = self.db.Symbol[mask]
        
    def split_binary_mask(self):
        ''' transform bit string, e.g. "5" or "2-3" to binary bit mask '''
        mask = self.db['Mask'].notnull()
        self.db.Mask[mask] = self.db.Mask[mask].transform(self.get_mask)
        self.db.style.format({'Mask': '{:08b}'})
        
    def explicit_renaming(self):
        ''' explicit renaming '''
        dictionary = {'1541PCLSB': 'PCLSB_1541',\
                      '2XSEL': 'SEL_2X'}
        self.db.Symbol.replace(dictionary, inplace = True)
        self.db.MaskName.replace(dictionary, inplace = True)

    def __init__(self, lines):
        ''' Read raw lines from iomap.txt and mangle into Pandas Dataframe '''
        self.db = pd.DataFrame(lines, columns=['System', 'Addresses', 'Symbol', 'Description'])
        self.initial_trim()
        self.split_address_range()
        self.detect_categories()
        self.remove_dashes()
        self.db = self.db[['System', 'Category', 'Lower', 'Upper', 'Symbol', 'Mask', 'Description']]
        self.make_long_names()
        self.name_masks()
        self.split_binary_mask()
        self.explicit_renaming()        
        self.db = self.db.drop_duplicates(subset=['LongName'])

db = ParseIomap(lines).db
db.to_csv('iomap.csv')
db.to_json('iomap.json')

In [ ]:
has_register_name = db.Mask.isnull() & db.Symbol.isnull()
register_names = db[has_register_name][['Lower', 'LongName']]

def func(row):
    a = register_names.LongName[grouped.Lower == row.Lower]
    if len(a) > 0:
        return a.values[0]
    return None

db['RegisterName'] = db.apply(func, axis=1)
db

In [ ]:
class FlatRustBySystem:
    ''' Flat rust constants grouped by Category (VIC, SID, etc) '''
    def write_mask(self, file, row):
        file.write(f'  /// {row.Description} [0x{row.Lower}: {row.Symbol}]\n')
        file.write(f'  pub const {row.LongName.upper()}_MASK: u8 = 0b{int(row.Mask):08b};\n\n')

    def write_address(self, file, row):
        if row.Symbol == '':
            return
        file.write(f'  /// {row.Description} [0x{row.Lower}: {row.Symbol}]\n')
        file.write(f'  pub const {row.LongName.upper()}: *mut u8 = (0x{row.Lower}) as *mut u8;\n\n')

    def write_item(self, file, row):
        if row.Mask != None:
            self.write_mask(file, row)
        else:
            self.write_address(file, row)
            
    def module_begin(self, file, name):
        file.write(f'pub mod {name.lower()} {{\n\n')

    def module_end(self, file):
        file.write('}\n\n')

    def create(self, file, db):
        no_upper_address = db['Upper'].isnull()
        has_symbol = db['Symbol'].notnull()
        grouped = db[no_upper_address & has_symbol].groupby('Category')
        for category, group in grouped:
            self.module_begin(file, category)
            for index, row in group.iterrows():
                self.write_item(file, row)
            self.module_end(file)
        return db

    
with open('iomap.rs', 'w') as file:
    FlatRustBySystem().create(file, db)


In [ ]:
def make_bitflags(db, address, name):
    ''' Generate rust bitflags for masks found for a given register, specified by it's address '''
    print(f'bitflags! {{\n\tpub struct {name}: u8 {{')
    for index, row in db.groupby('Lower').get_group(address).iterrows():
        if row.Mask == None:
            continue
        item = '\t\tconst {} = 0b{:08b};'.format(row.LongName.upper(), int(row.Mask))
        print(item)
    print('\t}\n}\n')
        
make_bitflags(db, 'D404', 'hej')

In [ ]:
class FlatCHeader:
    ''' Flat C style header with defines '''
        
    def write_mask(self, file, row):
        ''' Print C-header define '''
        mask = row.Mask
        symbol = '{}_{}_MASK'.format(row.Category, row.Symbol)
        file.write('/* {} (bit mask) */\n'.format(row.Description))
        file.write('#define {} {:08b}\n'.format(symbol, int(mask)))

    def write_address(self, file, row):
        ''' Print C-header define '''
        symbol = '{}_{}'.format(row.Category, row.Symbol)
        file.write('/** {} (address) */\n'.format(row.Description))
        file.write('#define {} 0x{}\n'.format(symbol, row.Lower))

    def write_item(self, file, row):
        if row.Mask != None:
            self.write_mask(file, row)
        else:
            self.write_address(file, row)

    def create(self, file, db):
        ''' Make a flat list of registers '''
        no_upper_address = db['Upper'].isnull()
        has_symbol = db['Symbol'].notnull()
        db = db[no_upper_address & has_symbol]
        for index, row in db.iterrows():
            self.write_item(file, row)
        return db

    
with open('iomap.h', 'w') as file:
    FlatCHeader().create(file, db)
